In [3]:
# Python ≥3.5 is required
import sys

import keras.datasets.mnist

print("Python: ", sys.version_info)
assert sys.version_info >= (3, 7)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.8 is required
import tensorflow as tf
print("TF version: ", tf.__version__)
# assert tf.__version__ >= "2.8"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Common imports
import numpy as np
import os
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow_datasets.core.utils import gcs_utils
gcs_utils._is_gcs_disabled = True
from tensorflow import keras


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

Python:  sys.version_info(major=3, minor=9, micro=19, releaselevel='final', serial=0)
sklearn version:  1.4.1.post1
TF version:  2.15.0
GPU installed:  True
1 Physical GPUs, 1 Logical GPUs


In [4]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "deep"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    savefig = plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
! pip install --upgrade tfds-nightly

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


## get the data(EMNIST)

In [10]:
import tensorflow_datasets as tfds

(train_data, valid_data, test_data), info = tfds.load("emnist/byclass", split=["train[:80%]", "train[80%:]", "test"], as_supervised=True, with_info=True)

In [11]:
print(f"size of train: {train_data.cardinality().numpy()}")
print(f"size of validation: {valid_data.cardinality().numpy()}")
print(f"size of test: {test_data.cardinality().numpy()}")

size of train: 558346
size of validation: 139586
size of test: 116323


In [12]:
num_class = info.features["label"].num_classes
print(num_class)

62


## prepare the data

In [13]:
def preprocess_resnet50(image, label):
    # Resize image to (32, 32, 1)
    image = tf.image.resize(image, size=(32, 32))
    # Convert grayscale to RGB by duplicating the single channel
    image = tf.image.grayscale_to_rgb(image)
    # Rescale pixel values to [0, 1]
    image = tf.cast(image, tf.float64) / 255.0
    return image, label

In [14]:
train_data = train_data.map(preprocess_resnet50)
valid_data = valid_data.map(preprocess_resnet50)
test_data = test_data.map(preprocess_resnet50)

In [15]:
batch_size = 32
train_data = train_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

## train / evaluate ResNet50

In [9]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("resNet50_checkpoint",
                                                         save_best_only=True)
run_index = 1 # 모델을 훈련할 때마다 증가합니다.
run_logdir = Path() / "resNet50Log" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [22]:
%load_ext tensorboard
%tensorboard --logdir=./resNet50Log

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

In [17]:
resNet = tf.keras.applications.ResNet50(classes=num_class, weights=None, input_shape=(32,32,3), include_top=False)
resNetv2 = tf.keras.applications.ResNet50V2(classes=num_class, weights=None, input_shape=(32,32,3))

resNetv2.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 18, 18, 64)           0         ['conv1_conv[0][0]']          
                                                                                         

In [19]:
optimizer = tf.keras.optimizers.Nadam()

[transfer learning](https://keras.io/guides/transfer_learning/) 참고

In [20]:
resNetv2.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')

In [21]:
history = resNetv2.fit(train_data, epochs=5, validation_data=valid_data, batch_size=2000, callbacks=callbacks)

Epoch 1/5


2024-05-15 11:48:48.751495: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-05-15 11:48:54.350763: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f97432f9130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-15 11:48:54.350807: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-05-15 11:48:54.368067: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1715741334.465604    3028 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


17449/17449 [==============================] - ETA: 0s - loss: 0.5629 - accuracy: 0.8112INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


17449/17449 [==============================] - 991s 55ms/step - loss: 0.5629 - accuracy: 0.8112 - val_loss: 0.4275 - val_accuracy: 0.8445
Epoch 2/5
17449/17449 [==============================] - ETA: 0s - loss: 0.4033 - accuracy: 0.8530INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


...

17449/17449 [==============================] - 1067s 61ms/step - loss: 0.4033 - accuracy: 0.8530 - val_loss: 0.3915 - val_accuracy: 0.8545
Epoch 3/5
17449/17449 [==============================] - ETA: 0s - loss: 0.3697 - accuracy: 0.8624INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


17449/17449 [==============================] - 984s 56ms/step - loss: 0.3697 - accuracy: 0.8624 - val_loss: 0.3844 - val_accuracy: 0.8574
Epoch 4/5
17448/17449 [============================>.] - ETA: 0s - loss: 0.3498 - accuracy: 0.8680INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


INFO:tensorflow:Assets written to: resNet50_checkpoint/assets


17449/17449 [==============================] - 906s 52ms/step - loss: 0.3498 - accuracy: 0.8680 - val_loss: 0.3792 - val_accuracy: 0.8584
Epoch 5/5
17449/17449 [==============================] - 892s 51ms/step - loss: 0.3402 - accuracy: 0.8706 - val_loss: 0.5823 - val_accuracy: 0.8592


(224,224,3) 으로 진행하니 epoch 당 40분이 걸림...
대신 최소 사이즈인 (32,32,3) 으로 진행해봄

# TODO:
1. callback 추가(earlystopping 등)
2. tensorboard로 model training 시각화(교재 참고) 
3. 지금은 (224,224,3) 으로 resize 해서 넣었는데, 매우 매우 비효율적이며 원본 이미지 역시 손상되는 방식으로 진행됨. 
4. ResNet에 weight을 주고 학습시킬 지, 그냥 base부터 시작할지

## save model

In [6]:
MODEL_PATH = Path() / "models" / "ResNet50"
MODEL_PATH.mkdir(parents=True, exist_ok=True)

In [24]:
resNetv2.save(str(MODEL_PATH) + "/test.keras")

## reload model

In [7]:
model = tf.keras.models.load_model(str(MODEL_PATH) + "/test.keras")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.legend(loc="lower left")  # 추가 코드
save_fig("keras_learning_curves_plot")  # 추가 코드
plt.show()